In [61]:
!pip3 install qrcode
!pip3 install python-resize-image


You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.9/bin/python3.9 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.9/bin/python3.9 -m pip install --upgrade pip' command.


In [3]:
import pandas as pd
import ssl
ssl._create_default_https_context = ssl._create_unverified_context


sheet_id = "1fTaZGNrpIC6ObWOA46TQ_eA6kR5E3hh36-CBIv0EKCY"
eunyu_gid = 0
matt_gid = 865169686

url_eunyu = f"https://docs.google.com/spreadsheets/d/{sheet_id}/export?gid={eunyu_gid}&format=csv"
url_matt = f"https://docs.google.com/spreadsheets/d/{sheet_id}/export?gid={matt_gid}&format=csv"

eunyu_df = pd.read_csv(url_eunyu)
matt_df = pd.read_csv(url_matt)

In [12]:
total_names = eunyu_df[['Name', 'Connection']][:-1].append(matt_df[['Name', 'Connection']][:-4])
total_names.fillna('Others', inplace=True)
total_names

/var/folders/kc/83g7q_n10f352lr0d32t57lw0000gn/T/ipykernel_83897/2889858979.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  total_names = eunyu_df[['Name', 'Connection']][:-1].append(matt_df[['Name', 'Connection']][:-4])


,Name,Connection
0,Bong Xin Yi,Hall friends
1,Yuki,Hall friends
2,Dion,Hall friends
3,Giwen,Hall friends
4,Jon San,Hall friends
...,...,...
153,Xue Min,LTA
154,Yanzhang,LTA
155,Gin,LTA
156,Hafiz,LTA


In [19]:
import pandas as pd

new_total_names = pd.read_clipboard(index_col='id')

new_total_names


,name,connection,isadmin,url,slug,language,attendance
id,,,,,,,
1,Bong Xin Yi,Temasek Hall,False,https://mattyubygrace.com/feed?userId=bong-xin...,bong-xin-yi-hall-friends,en,"{""church"": true, ""dinner"": false}"
2,Yuki,Temasek Hall,False,https://mattyubygrace.com/feed?userId=yuki-hal...,yuki-hall-friends,en,"{""church"": true, ""dinner"": true}"
3,Dion,Temasek Hall,False,https://mattyubygrace.com/feed?userId=dion-hal...,dion-hall-friends,en,"{""church"": true, ""dinner"": true}"
4,Giwen,Temasek Hall,False,https://mattyubygrace.com/feed?userId=giwen-ha...,giwen-hall-friends,en,"{""church"": true, ""dinner"": true}"
5,Jon San,Temasek Hall,False,https://mattyubygrace.com/feed?userId=jon-san-...,jon-san-hall-friends,en,"{""church"": true, ""dinner"": true}"
...,...,...,...,...,...,...,...
232,Xue Min,LTA,False,https://mattyubygrace.com/feed?userId=xue-min-lta,xue-min-lta,en,"{""church"": true, ""dinner"": false}"
233,Yanzhang,LTA,False,https://mattyubygrace.com/feed?userId=yanzhang...,yanzhang-lta,en,"{""church"": true, ""dinner"": false}"
234,Gin,LTA,False,https://mattyubygrace.com/feed?userId=gin-lta,gin-lta,en,"{""church"": true, ""dinner"": false}"


In [22]:
def convert_to_snake_case(name, connection):
    # Convert the string to lowercase
    lowercase_string = name.lower()
    connection_string = connection.lower()

    # Replace spaces with hyphens
    hyphenated_string = f'{lowercase_string}-{connection_string}'.replace(' ', '-')

    # Convert the hyphenated string to snake case
    snake_case_string = hyphenated_string.replace(' ', '_')

    

    return snake_case_string


base_url = 'https://mattyubygrace.com/feed?userId='
slug = []
url = []

romaja_mapper = {
    "아빠": "appa",
    "엄마": "eomma",
    "언니": "eonni",
    "황장로님": "hwangjanglonim",
    "주권사님": "jugwonsanim",
    "작은 아빠": "jageun appa",
    "큰 아빠": "keun appa",
    "작은 이모": "jageun emo",
    "화영": "Hwayoung",
    "큰이모": "keun emo",
    "휘진": "Hwijin",
    "태영": "Taeyoung"
}
for idx, row in new_total_names.iterrows():
    name = romaja_mapper.get(row['name'], row['name'])
    slug.append(convert_to_snake_case(name, row['connection']))
    url.append(f"{base_url}{convert_to_snake_case(name, row['connection'])}")

new_total_names['url'] = url
new_total_names['slug'] = slug

new_total_names.to_clipboard()


In [23]:
# Generate QR codes

import qrcode

qr_codes = []

def make_qr_code(link):
  qr = qrcode.QRCode(
      version=8,
      error_correction=qrcode.constants.ERROR_CORRECT_L,
      box_size=6,
      border=0,
  )
  qr.add_data(link)

  
  qr.make(fit=True)
  
  return qr.make_image(fill_color="rgb(198, 139, 87)", back_color="white")


for idx, row in new_total_names.iterrows():
  img = make_qr_code(row['url'])
  qr_codes.append(img)

print(len(qr_codes))


236


In [24]:
from PIL import Image, ImageDraw, ImageFont, ImageOps

cormorant_font = './template/Cormorant-VariableFont_wght.ttf'

def create_front(name):
    front = Image.open('./template/front.png')
    canvas = ImageDraw.Draw(front)
    canvas.text((741, 430), name, font=ImageFont.truetype(
        cormorant_font, 200), fill=(243, 235, 225), anchor="mm")
    return front

def create_back(qr_code, link):
    back = Image.open('./template/back.png')
    canvas = ImageDraw.Draw(back)
    canvas.text((340, 650), 'https://mattyubygrace.com', font=ImageFont.truetype(cormorant_font, 50),
                fill=(243, 235, 225), anchor="mm")
    back.paste(qr_code, (190, 300))
    
    return back


images = []
for idx, row in new_total_names.iterrows():
    front = create_front(row['name'])
    back = create_back(qr_codes[idx - 1], row['url'])

    base_name = f"{row['name']}"

    images.append(front.convert("RGB"))
    images.append(back.convert("RGB"))
    # front.save(f'output/{base_name}-front.png')
    # back.save(f'output/{base_name}-back.png')

    images[0].save("output/compile.pdf", save_all=True, append_images=images[1:])


In [20]:
import pandas as pd
msg_df = pd.read_clipboard(sep="\t")

def treat_messages(msg):
    if type(msg) is float: return ''
    paras = msg.split('\n')
    paras = [f'<p>{para}</p>' for para in paras]
    return ''.join(paras)

msg_df['treated_msg'] = msg_df['message'].map(treat_messages)

msg_df.to_csv('output.csv')